In [1]:
import sympy as smp
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from itertools import product

In [2]:
smp.init_printing(use_unicode=True)

In [3]:
class FileSystemManager:
    def __init__(self):
        pass
    def open_file(self, text, filepath, flag):
        """
        Opens a file in read or write mode.
        
        :param text: Text to write to the file if flag is 'w'
        :param filepath: Path of the file to open
        :param flag: 'r' to read, 'w' to write
        :return: File content if reading, None if writing
        """
        try:
            if flag == 'r':
                with open(filepath, 'r') as f:
                    data = f.read()
                    return data  # return instead of print
            elif flag == 'w':
                with open(filepath, 'w') as f:
                    f.write(text)
            else:
                raise ValueError("Unsupported flag. Use 'r' for read or 'w' for write.")
        except FileNotFoundError:
            print(f"Error: File {filepath} not found.")
        except IOError as e:
            print(f"Error accessing file {filepath}: {e}")

    def latex(self):
        pass

In [4]:
class MagmaCalculator:
    """
    A class to interact with the Magma Calculator webpage and submit code for evaluation.
    
    Attributes:
    ----------
    output_file : str
        The name of the file where the output from the Magma Calculator will be saved.
    driver : webdriver.Chrome
        A Chrome web driver instance to automate browser interaction.
    url : str
        The URL of the Magma Calculator page.
    
    Methods:
    -------
    submit_code(code):
        Submits the given Magma code to the calculator and saves the result to a file.
    
    close():
        Closes the browser session.
    """
    
    def __init__(self, output_file="MagmaCalcResult"):
        """
        Initializes the MagmaCalculator with the given output file name.
        
        Parameters:
        ----------
        output_file : str, optional
            The name of the file where the result will be saved (default is "output.txt").
        """
        self.url = "http://magma.maths.usyd.edu.au/calc/"
        self.output_file = output_file
        self.file_manager = FileSystemManager()

    def submit_code(self, code):
        """
        Submits the given code to the Magma Calculator and saves the result to the output file.
        
        Parameters:
        ----------
        code : str
            The Magma code to be submitted for evaluation.
        
        Actions:
        -------
        - Opens the Magma Calculator webpage.
        - Finds the input box and enters the code.
        - Clicks the submit button.
        - Waits for the result to load.
        - Retrieves the result and writes it to the specified output file.
        """
        driver = webdriver.Chrome()
        try:
            driver.get(self.url)
            
            input_box = driver.find_element(By.ID, "input")
            input_box.clear()
            input_box.send_keys(code)
            
            submit_button = driver.find_element(By.XPATH, "//input[@value='Submit']")
            submit_button.click()
            
            time.sleep(5)
            
            result_element = driver.find_element(By.ID, "result")
            result_text = result_element.get_attribute('value')
            
            self.file_manager.open_file(result_text, self.output_file, 'w')
        finally:
            driver.quit()

In [5]:
class FlynnLeprevost:
    def __init__(self, m=27, g=2, inf_count=1, range_limit=30, curves_path="foundCurves.txt", magma_result_path="MagmaCalcResult"):
        """"""
        self.x, self.y = smp.symbols('x y')
        self.f = smp.symbols('f')
        self.V1, self.V2, self.W1, self.W2 = smp.symbols('V1 V2 W1 W2') 
        self.w = smp.symbols('w')
        self.m = m
        self.g = g
        self.range_limit = range_limit
        if inf_count == 1:
            self.deg = 2 * g + 1
        elif inf_count == 2:
            self.deg = 2 * g + 2
        else:
            raise ValueError("inf_count must be 1 or 2")
        self.constraints = [self.deg, self.deg + 3*self.g]
        self.coeffs = self._get_coeffs()
        self.curves_path = curves_path
        self.magma_calc = MagmaCalculator(magma_result_path)
        self.file_manager = FileSystemManager()

    def factor_coeffs(self, f, val):
        """
        Factors the coefficients of a polynomial and computes the sum of 
        each factored coefficient multiplied by a power of the given value.
        
        :param f: The polynomial expression whose coefficients will be factored.
        :param val: The symbol or value to use as the base for powers in the sum.
        
        :return: The sum of the factored coefficients of the polynomial `f`,
                each multiplied by the corresponding power of `val`.
                
        Example:
        If f = 2*x**2 + 3*x + 4 and val = x, the function will factor each 
        coefficient (if possible) and return the sum 4 + 3*x + 2*x**2.
        """
        return sum(smp.factor(coeff)*val**i for i, coeff in enumerate(reversed(smp.Poly(f, val).all_coeffs())))

    def torsion_point_order_divisor(self, a1, b1, a2, b2):
        """Calculate the divisor of torsion point order using the determinant."""
        matrix = smp.Matrix([[a1, b1], [a2, b2]])
        m = abs(matrix.det())
        return m
    
    def check_order(self, equation):
        """Function that helps to check an order of possible torison points of given function"""
        code = f"""
        P<x> := PolynomialRing(Rationals());
        C1 := HyperellipticCurve({equation});
        J1 := Jacobian(C1);
        TorsionSubgroup(J1);
        ClebschInvariants(C1);
        IgusaClebschInvariants(C1);
        IgusaInvariants(C1);"""
        self.magma_calc.submit_code(code, self.magma_calc.output_file)

    def divisor_deg(self, deg_V_j, deg_W_j):
        """
        Checks the divisor degree of given function
        deg*: degree of divisor
        deg: degree  of polynomial 
        deg* u_j = max(2 deg(V_j), 2 deg(W_j) + 2g + 1)
        """
        return max(2* smp.degree(deg_V_j, self.x), 2 * smp.degree(deg_W_j, self.x) + self.deg)

    def in_constraints(self, coeffs_set):
        return (abs(coeffs_set[0]) + abs(coeffs_set[1]) <= abs(self.constraints[1])) and (abs(coeffs_set[0]) + abs(coeffs_set[1]) >= abs(self.constraints[0])) and (abs(coeffs_set[2]) + abs(coeffs_set[3]) <= abs(self.constraints[1])) and (abs(coeffs_set[2]) + abs(coeffs_set[3]) >= abs(self.constraints[0]))
    
    def coeffs_restrictions(self, coeffs_set):
        return self.torsion_point_order_divisor(*coeffs_set) == self.m and self.in_constraints(coeffs_set)

    # def get_coeffs(self):
    #     values = range(1, self.range_limit)
    #     final_coeffs = []

    #     coeffs_sets = product(values, repeat=4)

    #     for coeffs_set in coeffs_sets:
    #         if self.coeffs_restrictions(coeffs_set):
    #             final_coeffs.append(coeffs_set)

    #     return final_coeffs
    
    def _get_coeffs(self):
        values = [i for i in range(1, self.range_limit)]
        values_last =[i for i in range(-self.range_limit,0)] 
        final_coeffs = []
        coeffs_sets = [[a1, b1, a2, b2] for a1 in values for b1 in values for a2 in values for b2 in values_last]
        for coeffs_set in coeffs_sets:
            if self.coeffs_restrictions(coeffs_set):
                final_coeffs.append(coeffs_set)

        return final_coeffs
    
    def get_system_information(self, coeffs_set):  
        a1,b1,a2,b2 = abs(coeffs_set[0]),abs(coeffs_set[1]),abs(coeffs_set[2]),abs(coeffs_set[3])

        deg_V1 = int(np.floor((a1 + b1) / 2))
        deg_V2 = int(np.floor((a2 + b2) / 2))
        deg_W1 = int(np.floor((a1 + b1 - self.deg) / 2))
        deg_W2 = int(np.floor((a2 + b2 - self.deg) / 2))

        alpha, beta = min(a1, a2), min(b1, b2)
        gamma = smp.symbols('gamma')
        a = smp.symbols('a')

        cum_deg_left = 2 * max(deg_V1 + deg_W2, deg_V2 + deg_W1)
        cum_deg_right = alpha + beta + max(2*deg_W2 + (a1 - alpha) + (b1 - beta), 2*deg_W1 + (a2 - alpha) + (b2 - beta))
        
        deg_w = int(cum_deg_left/2 - alpha)

        if (a1 + b1) % 2 == 0 and (a2 + b2) % 2 == 0: 
            lc_V1 = 1
            lc_V2 = 1
            lc_W1 = 1
            lc_W2 = a
            lc_f = -gamma
        elif (a1 + b1) % 2 == 0 and (a2 + b2) % 2 != 0:
            lc_V1 = 1
            lc_V2 = a
            lc_W1 = 1
            lc_W2 = 1
            lc_f = -gamma
        elif (a1 + b1) % 2 != 0 and (a2 + b2) % 2 != 0:
            lc_V1 = 1
            lc_V2 = a
            lc_W1 = 1
            lc_W2 = 1
            lc_f = -gamma
        elif (a1 + b1) % 2 != 0 and (a2 + b2) % 2 == 0:
            lc_V1 = a
            lc_V2 = 1    
            lc_W1 = 1
            lc_W2 = 1
            lc_f = -gamma
            
        system = {
            'coeffs_set': coeffs_set,
            'deg(V1)': deg_V1,
            'deg(V2)': deg_V2,
            'deg(W1)': deg_W1,
            'deg(W2)': deg_W2,  
            'deg(w)' : deg_w,
            'cum_deg_left': cum_deg_left,
            'cum_deg_right': cum_deg_right, 
            'lc(W1)': lc_W1,
            'lc(W2)': lc_W2,
            'lc(V1)': lc_V1,
            'lc(V2)': lc_V2,
            'lc(f)': lc_f,
        }
        return system

    def get_systems_information(self):
        coeffs_sets = self.coeffs
        systems_information = []
        for coeffs_set in coeffs_sets:
            systems_information.append(self.get_system_information(coeffs_set))

        return systems_information
    
    def get_system(self, coeffs_set):
        system_information = self.get_system_information(coeffs_set)

        left_part_1 = self.V1**2 - self.W1**2 * self.f
        left_part_2 = self.V2**2 - self.W2**2 * self.f

        gamma_1 = smp.symbols('gamma_1')  
        gamma_2 = smp.symbols('gamma_2')
        power_x = smp.symbols('n')     
        power_x_minus_1 = smp.symbols('m')  

        equation_1 = gamma_1 * self.x**power_x * (self.x - 1)**power_x_minus_1 
        equation_2 = gamma_2 * self.x**power_x * (self.x - 1)**power_x_minus_1 

        right_part_1 = equation_1.subs({power_x: abs(coeffs_set[0]), power_x_minus_1: abs(coeffs_set[1])})
        right_part_2 = equation_2.subs({power_x: abs(coeffs_set[2]), power_x_minus_1: abs(coeffs_set[3])})

        eq_1 = smp.Eq(left_part_1, right_part_1)
        eq_2 = smp.Eq(left_part_2, right_part_2)

        system = {
            'coeffs_set': system_information ['coeffs_set'],
            'deg(V1)': system_information ['deg(V1)'],
            'deg(V2)': system_information ['deg(V2)'],
            'deg(W1)': system_information ['deg(W1)'],
            'deg(W2)': system_information ['deg(W2)'],
            'cum_deg_left': system_information ['cum_deg_left'],
            'cum_deg_right': system_information ['cum_deg_right'],
            'eq_1': eq_1,
            'eq_2': eq_2,
        }
        return system

    def get_systems(self):
        final_systems = []
        coeffs_set = self.coeffs
        for coeffs in coeffs_set:
            system = self.get_system(coeffs)
            final_systems.append(system)

        return final_systems

    def get_equation(self, coeffs_set):
        system_information = self.get_system_information(coeffs_set)

        gamma_1 = smp.symbols('gamma')  
        gamma_2 = smp.symbols('gamma') 

        a1,b1,a2,b2 = abs(coeffs_set[0]),abs(coeffs_set[1]),abs(coeffs_set[2]),abs(coeffs_set[3])

        alpha, beta = min(a1, a2), min(b1, b2)

        if (a1 + b1) % 2 == 0 and (a2 + b2) % 2 == 0: 
            gamma_1 = 1
            gamma_2 = 1
        elif (a1 + b1) % 2 == 0 and (a2 + b2) % 2 != 0:
            gamma_1 = 1
        elif (a1 + b1) % 2 != 0 and (a2 + b2) % 2 == 0:
            gamma_2 = 1

        left_part = (self.V1 * self.W2 - self.V2 * self.W1)*(self.V1 * self.W2 + self.V2 * self.W1)
        right_part = self.x**alpha * (self.x - 1)**beta * (gamma_1 * self.W2**2 * self.x**(a1 - alpha) * (self.x - 1)**(b1 - beta)
                                                         - gamma_2 * self.W1**2 * self.x**(a2 - alpha) * (self.x - 1)**(b2 - beta))

        equation = smp.Eq(left_part, right_part)

        equation = {
            'coeffs_set': system_information ['coeffs_set'],
            'deg(V1)': system_information ['deg(V1)'],
            'deg(V2)': system_information ['deg(V2)'],
            'deg(W1)': system_information ['deg(W1)'],
            'deg(W2)': system_information ['deg(W2)'],
            'deg(w)' : system_information ['deg(w)'], 
            'cum_deg_left': system_information ['cum_deg_left'],
            'cum_deg_right': system_information ['cum_deg_right'],
            'equation': equation,
            'lc(W1)': system_information ['lc(W1)'],
            'lc(W2)': system_information ['lc(W2)'],
            'lc(V1)': system_information ['lc(V1)'],
            'lc(V2)': system_information ['lc(V2)'],
            'lc(f)': system_information ['lc(f)'],
        }
        return equation

    def get_equations(self):
        simpyified_equations = []
        for coeffs_set in self.coeffs:
            simpyified_equations.append(self.get_equation(coeffs_set))
        return simpyified_equations
    
    def get_system_fe(self, coeffs_set):
        pass 

    def get_systems_fe(self):
        systems = []
        for coeffs_set in self.coeffs:
            systems.append(self.get_system_fe(coeffs_set))
        return systems

In [6]:
model = FlynnLeprevost()
model.coeffs

[[1, 4, 6, -3], [1, 5, 4, -7], [1, 5, 5, -2], [1, 6, 4, -3], [1, 7, 3, -6], [1 ↪

↪ , 8, 3, -3], [1, 9, 2, -9], [1, 10, 2, -7], [2, 3, 5, -6], [2, 3, 7, -3], [2 ↪

↪ , 5, 3, -6], [2, 5, 5, -1], [2, 7, 1, -10], [2, 7, 3, -3], [2, 9, 1, -9], [3 ↪

↪ , 2, 3, -7], [3, 2, 6, -5], [3, 3, 1, -8], [3, 3, 2, -7], [3, 3, 3, -6], [3, ↪

↪  3, 4, -5], [3, 3, 5, -4], [3, 3, 6, -3], [3, 3, 7, -2], [3, 3, 8, -1], [3,  ↪

↪ 4, 3, -5], [3, 4, 6, -1], [3, 5, 3, -4], [3, 6, 1, -7], [3, 6, 2, -5], [3, 6 ↪

↪ , 3, -3], [3, 6, 4, -1], [3, 7, 3, -2], [4, 1, 3, -6], [4, 3, 1, -6], [4, 3, ↪

↪  5, -3], [4, 5, 3, -3], [4, 7, 1, -5], [5, 1, 2, -5], [5, 1, 7, -4], [5, 2,  ↪

↪ 1, -5], [5, 2, 6, -3], [5, 3, 4, -3], [5, 4, 3, -3], [5, 6, 2, -3], [6, 1, 3 ↪

↪ , -4], [6, 3, 1, -4], [6, 3, 3, -3], [6, 3, 5, -2], [6, 3, 7, -1], [6, 5, 3, ↪

↪  -2], [7, 1, 6, -3], [7, 2, 3, -3], [7, 2, 10, -1], [7, 3, 2, -3], [7, 4, 5, ↪

↪  -1], [8, 1, 3, -3], [9, 1, 9, -2], [9, 2, 9, -1], [10, 1, 7, -2]]

In [7]:
systems_information = model.get_systems_information()
systems = model.get_systems()
simpyified_equations = model.get_equations()

In [8]:
cur_sys = systems[-2]

In [9]:
cur_sys['eq_1']

  2     2         9        2
V₁  - W₁ ⋅f = γ₁⋅x ⋅(x - 1) 

In [10]:
cur_sys['eq_2']

  2     2         9        
V₂  - W₂ ⋅f = γ₂⋅x ⋅(x - 1)

In [11]:
simpyified_equations

[{'coeffs_set': [1, 4, 6, -3],
  'deg(V1)': 2,
  'deg(V2)': 4,
  'deg(W1)': 0,
  'deg(W2)': 2,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**3*(-W1**2*gamma*x**5 + W2**2*gamma*(x - 1))),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [1, 5, 4, -7],
  'deg(V1)': 3,
  'deg(V2)': 5,
  'deg(W1)': 0,
  'deg(W2)': 3,
  'deg(w)': 5,
  'cum_deg_left': 12,
  'cum_deg_right': 12,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**5*(-W1**2*gamma*x**3*(x - 1)**2 + W2**2)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [1, 5, 5, -2],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 0,
  'deg(W2)': 1,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 8,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**2*(-W1**2*gamma*x**4 + W2**2*(x - 1)**3)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'l

In [12]:
def filter_by_w(deg_w, simpyified_equations):
    filtered_sys = []
    for sy in simpyified_equations:
        if sy['deg(w)'] == deg_w:
            filtered_sys.append(sy)
    return filtered_sys

In [13]:
def filter_by_w(deg_w, simpyified_equations):
    filtered_sys = []
    for sy in simpyified_equations:
        if sy['deg(w)'] == deg_w:
            filtered_sys.append(sy)
    return filtered_sys

In [14]:
filtered = filter_by_w(5, simpyified_equations)

In [15]:
filtered

[{'coeffs_set': [1, 5, 4, -7],
  'deg(V1)': 3,
  'deg(V2)': 5,
  'deg(W1)': 0,
  'deg(W2)': 3,
  'deg(w)': 5,
  'cum_deg_left': 12,
  'cum_deg_right': 12,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**5*(-W1**2*gamma*x**3*(x - 1)**2 + W2**2)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [1, 7, 3, -6],
  'deg(V1)': 4,
  'deg(V2)': 4,
  'deg(W1)': 1,
  'deg(W2)': 2,
  'deg(w)': 5,
  'cum_deg_left': 12,
  'cum_deg_right': 12,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**6*(-W1**2*gamma*x**2 + W2**2*(x - 1))),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [3, 6, 1, -7],
  'deg(V1)': 4,
  'deg(V2)': 4,
  'deg(W1)': 2,
  'deg(W2)': 1,
  'deg(w)': 5,
  'cum_deg_left': 12,
  'cum_deg_right': 12,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**6*(-W1**2*(x - 1) + W2**2*gamma*x**2)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': a,
  'lc(V2)': 1,
  'lc(f)'

In [16]:
filtered = filter_by_w(4, simpyified_equations)

In [17]:
filtered

[{'coeffs_set': [1, 8, 3, -3],
  'deg(V1)': 4,
  'deg(V2)': 3,
  'deg(W1)': 2,
  'deg(W2)': 0,
  'deg(w)': 4,
  'cum_deg_left': 10,
  'cum_deg_right': 10,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**3*(-W1**2*x**2 + W2**2*gamma*(x - 1)**5)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': a,
  'lc(V2)': 1,
  'lc(f)': -gamma},
 {'coeffs_set': [3, 3, 1, -8],
  'deg(V1)': 3,
  'deg(V2)': 4,
  'deg(W1)': 0,
  'deg(W2)': 2,
  'deg(w)': 4,
  'cum_deg_left': 10,
  'cum_deg_right': 10,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**3*(-W1**2*gamma*(x - 1)**5 + W2**2*x**2)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma}]

In [18]:
filtered = filter_by_w(3, simpyified_equations)

In [19]:
filtered

[{'coeffs_set': [1, 4, 6, -3],
  'deg(V1)': 2,
  'deg(V2)': 4,
  'deg(W1)': 0,
  'deg(W2)': 2,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**3*(-W1**2*gamma*x**5 + W2**2*gamma*(x - 1))),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [1, 5, 5, -2],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 0,
  'deg(W2)': 1,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 8,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**2*(-W1**2*gamma*x**4 + W2**2*(x - 1)**3)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [1, 6, 4, -3],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 1,
  'deg(W2)': 1,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**3*(-W1**2*gamma*x**3 + W2**2*gamma*(x - 1)**3)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,


In [20]:
filtered = filter_by_w(2, simpyified_equations)

In [21]:
filtered

[{'coeffs_set': [2, 5, 5, -1],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 1,
  'deg(W2)': 0,
  'deg(w)': 2,
  'cum_deg_left': 8,
  'cum_deg_right': 8,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**2*(x - 1)*(-W1**2*x**3 + W2**2*gamma*(x - 1)**4)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': a,
  'lc(V2)': 1,
  'lc(f)': -gamma},
 {'coeffs_set': [3, 2, 3, -7],
  'deg(V1)': 2,
  'deg(V2)': 5,
  'deg(W1)': 0,
  'deg(W2)': 2,
  'deg(w)': 2,
  'cum_deg_left': 10,
  'cum_deg_right': 10,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**3*(x - 1)**2*(-W1**2*(x - 1)**5 + W2**2*gamma)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': a,
  'lc(V2)': 1,
  'lc(f)': -gamma},
 {'coeffs_set': [3, 2, 6, -5],
  'deg(V1)': 2,
  'deg(V2)': 5,
  'deg(W1)': 0,
  'deg(W2)': 3,
  'deg(w)': 2,
  'cum_deg_left': 10,
  'cum_deg_right': 11,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**3*(x - 1)**2*(-W1**2*gamma*x**3*(x - 1)**3 + W2**2*gamma)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a

In [22]:
filtered = filter_by_w(1, simpyified_equations)

In [23]:
filtered

[{'coeffs_set': [3, 4, 6, -1],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 1,
  'deg(W2)': 1,
  'deg(w)': 1,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**3*(x - 1)*(-W1**2*gamma*x**3 + W2**2*gamma*(x - 1)**3)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [3, 6, 4, -1],
  'deg(V1)': 4,
  'deg(V2)': 2,
  'deg(W1)': 2,
  'deg(W2)': 0,
  'deg(w)': 1,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**3*(x - 1)*(-W1**2*gamma*x + W2**2*gamma*(x - 1)**5)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [4, 1, 3, -6],
  'deg(V1)': 2,
  'deg(V2)': 4,
  'deg(W1)': 0,
  'deg(W2)': 2,
  'deg(w)': 1,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**3*(x - 1)*(-W1**2*gamma*(x - 1)**5 + W2**2*gamma*x)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': 

In [24]:
filtered = filter_by_w(0, simpyified_equations)

In [25]:
len(filtered)

6

In [26]:
filtered

[{'coeffs_set': [5, 2, 6, -3],
  'deg(V1)': 3,
  'deg(V2)': 4,
  'deg(W1)': 1,
  'deg(W2)': 2,
  'deg(w)': 0,
  'cum_deg_left': 10,
  'cum_deg_right': 11,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**5*(x - 1)**2*(-W1**2*gamma*x*(x - 1) + W2**2*gamma)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [6, 3, 5, -2],
  'deg(V1)': 4,
  'deg(V2)': 3,
  'deg(W1)': 2,
  'deg(W2)': 1,
  'deg(w)': 0,
  'cum_deg_left': 10,
  'cum_deg_right': 11,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**5*(x - 1)**2*(-W1**2*gamma + W2**2*gamma*x*(x - 1))),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [6, 3, 7, -1],
  'deg(V1)': 4,
  'deg(V2)': 4,
  'deg(W1)': 2,
  'deg(W2)': 1,
  'deg(w)': 0,
  'cum_deg_left': 12,
  'cum_deg_right': 12,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**6*(x - 1)*(-W1**2*x + W2**2*gamma*(x - 1)**2)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': a,
  'lc(V2)': 1,


In [27]:
def get_coeffs_conditions(v, p):
    """
    Compute the symbolic coefficients of q(x) such that v(x) = p(x) * q(x).
    
    Parameters:
    v (smp.Poly): The dividend polynomial v(x).
    p (smp.Poly): The divisor polynomial p(x).
    
    Returns:
    dict or str: A dictionary of q(x) coefficients {q_i: expression} or a message.
    """
    if not isinstance(v, smp.Poly) or not isinstance(p, smp.Poly):
        raise ValueError("Both v and p must be sympy.Poly objects.")

    v_coeffs = v.all_coeffs()[::-1]  
    p_coeffs = p.all_coeffs()[::-1]  

    n = v.degree()
    m = p.degree()
    k = n - m 
    
    q_coeffs = list(smp.symbols(f'q:{k+1}'))

    if m == 0 and p_coeffs[0] == 0:
        return "Invalid p(x): Leading coefficient cannot be zero."
    if n < m:
        return "Degree of v(x) is less than degree of p(x); division not possible."
   
    p_coeffs += [0] * (n - m)
    q_coeffs += [0] * (n - k)

    zero = smp.Rational(0)
    eq_dict = {} 

    for i in range(0, n + 1):
        equation = zero
        for j in range(0, i+1):
            equation += p_coeffs[j] * q_coeffs[i-j]

        eq_i = smp.Eq(equation, v_coeffs[i])
        eq_dict[v_coeffs[i]] = eq_i

    return eq_dict

In [28]:
x = smp.symbols('x')
deg_v = 6
deg_p = 2

v = smp.symbols(f'v:{deg_v + 1}')
v = smp.Poly(sum(v[i] * x**i for i in range(deg_v,-1,-1)), x)

p= smp.symbols(f'p:{deg_p + 1}')
p = smp.Poly(sum(p[i] * x**i for i in range(deg_p,-1,-1)), x)

eqs = get_coeffs_conditions(v, p)

print(f'v : {v}')
print(f'p : {p}')
eqs

v : Poly(v6*x**6 + v5*x**5 + v4*x**4 + v3*x**3 + v2*x**2 + v1*x + v0, x, domain='ZZ[v0,v1,v2,v3,v4,v5,v6]')
p : Poly(p2*x**2 + p1*x + p0, x, domain='ZZ[p0,p1,p2]')


{v₀: p₀⋅q₀ = v₀, v₁: p₀⋅q₁ + p₁⋅q₀ = v₁, v₂: p₀⋅q₂ + p₁⋅q₁ + p₂⋅q₀ = v₂, v₃: p ↪

↪ ₀⋅q₃ + p₁⋅q₂ + p₂⋅q₁ = v₃, v₄: p₀⋅q₄ + p₁⋅q₃ + p₂⋅q₂ = v₄, v₅: p₁⋅q₄ + p₂⋅q₃ ↪

↪  = v₅, v₆: p₂⋅q₄ = v₆}

In [29]:
def solve_coeff_conditions(eqs_dict):
    """
    Solve the system of equations from divisibility conditions for parameters.
    
    Parameters:
    eqs_dict (dict): A dictionary of equations {coeff_i: Eq}.
    
    Returns:
    dict or str: Solutions for the parameters or a message if no solution exists.
    """
    equations = list(eqs_dict.values())
    
    symbols = set()
    for eq in equations:
        symbols.update(eq.free_symbols)
    
    solution = smp.solve(equations, symbols, dict=True)
    
    if not solution:
        return "No solution found for the given conditions."
    
    return solution

In [30]:
# Example Application:
x = smp.Symbol('x')

# Define w(x) as a polynomial of degree 4
w_coeffs = smp.symbols('w0 w1 w2 w3 w4')
w = smp.Poly(sum(w_coeffs[i] * x**i for i in range(5)), x)

# Define eq_2 using symbolic coefficients gamma and x0
gamma, x0 = smp.symbols('gamma x0')
eq_2 = smp.Poly(((x - 1) * (x - x0)**2 * x - gamma * (x - 1)**3), x)

# Generate conditions for divisibility
conditions = get_coeffs_conditions(eq_2, w)

# Solve for parameters
solutions = solve_coeff_conditions(conditions)

# Display results
print("Conditions:")
for coeff, eq in conditions.items():
    print(f"{coeff}: {eq}")

print("\nSolutions:")
print(solutions)

Conditions:
gamma: Eq(q0*w0, gamma)
-3*gamma - x0**2: Eq(q0*w1, -3*gamma - x0**2)
3*gamma + x0**2 + 2*x0: Eq(q0*w2, 3*gamma + x0**2 + 2*x0)
-gamma - 2*x0 - 1: Eq(q0*w3, -gamma - 2*x0 - 1)
1: Eq(q0*w4, 1)

Solutions:
[{gamma: -(w3 + 2*w4*x0 + w4)/w4, q0: 1/w4, w0: -w3 - 2*w4*x0 - w4, w1: 3*w3 - w4*x0**2 + 6*w4*x0 + 3*w4, w2: -3*w3 + w4*x0**2 - 4*w4*x0 - 3*w4}]


In [31]:
def subs_param(eq, params, param_num=0):
    """
    Substitute parameters in the equation.
    
    Parameters:
    eq (smp.Poly): The equation to substitute parameters in.
    params (dict): A dictionary of parameters {param_i: value}.
    
    Returns:
    smp.Poly: The equation with parameters substituted.
    """
    if not isinstance(eq, smp.Poly):
        raise ValueError("eq must be a sympy.Poly object.")
    
    return eq.subs(params[param_num])

In [32]:
eq_2 = subs_param(eq_2, solutions, 0)

In [33]:
smp.div(eq_2, w)

(Poly(0, x, x0, w0, w1, w2, w3, w4, 1/w4, domain='ZZ'), Poly(x**4 + x**3*w3*(1 ↪

↪ /w4) + x**2*x0**2 - 4*x**2*x0 - 3*x**2*w3*(1/w4) - 3*x**2 - x*x0**2 + 6*x*x0 ↪

↪  + 3*x*w3*(1/w4) + 3*x - 2*x0 - w3*(1/w4) - 1, x, x0, w0, w1, w2, w3, w4, 1/ ↪

↪ w4, domain='ZZ'))